In [1]:
import pandas as pd
import re
from itertools import chain

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [2]:
def import_features():
    features = pd.read_excel('phoneme data//features.xlsx', index_col=0).T
    print('Inventory:')
    print(', '.join([c for c in features.columns]))
    
    return features

In [3]:
def import_ipa():
    ipa_chart = pd.read_excel('phoneme data//IPA chart.xlsx', index_col=0).T.reset_index(drop=True)
    print('Inventory:')
    print(', '.join([c for c in ipa_chart.columns]))
    
    return ipa_chart

In [4]:
def import_vowels():
    vowels = pd.read_excel('phoneme data//vowel chart.xlsx', index_col=0).T.reset_index(drop=True)
    vowels
    print('Inventory:')
    print(', '.join([c for c in vowels.columns]))
    
    return vowels

In [5]:
def build_letter_list(inventory):
    
    group = []
    letter = None
    
    while letter != '0':
        letter = input('Enter a letter(0 to exit): ')
        if letter == '0':
            break
        elif letter not in inventory:
            print('Not in data.')
            continue
        else:
            group.append(letter.strip().lower())
    
    return group

In [6]:
# Compares groups of phones and gives similar features within groups but different across groups
# to just get commonalities of a group, enter '0' as the first letter in Group2
def compare_groups(choice):    
    
    if choice == '1':
        features = import_features()
    
    elif choice == '2':
        features = import_ipa()
    
    elif choice == '3':
        features = import_vowels()
        
    print('Group 1:')    
    group1 = build_letter_list(features.columns)
    
    if len(group1) < 1:
        return None
    
    print('\nGroup2:')
    group2 = build_letter_list(features.columns)

    group1_list = [(a, list(set(b))[0]) for a, b in features[group1].iterrows() if len(set(b)) == 1]
    if len(group1_list) < 1:
        print('No similarities')
        return None
    feature, value = zip(*group1_list)
    group1_similar = pd.Series(value, feature, name=', '.join(group1))
    
    if len(group2) > 0:        
        group2_list = [(a, list(set(b))[0]) for a, b in features[group2].iterrows() if len(set(b)) == 1]
        if len(group2_list) < 1:
            print('No similarities')
            return None
        
        feature, value = zip(*group2_list)
        group2_similar = pd.Series(value, feature, name=', '.join(group2))
    
        different_features = []

        for a, b in pd.concat([group1_similar, group2_similar], axis=1).iterrows():
            if a in group1_similar.index and a in group2_similar.index:
                if b[0] != b[1] and isinstance(b[0], str) and isinstance(b[1], str):
                    different_features.append([a, b[0], b[1]])

        compare = pd.DataFrame(different_features, 
                               columns=['features', group1_similar.name, group2_similar.name]).set_index('features')
        
        
        return compare
            
    else:
        group1_similar = pd.DataFrame(group1_similar)
        return group1_similar

In [ ]:
inputting = True

print('Welcome to my phonologizer!')
print('It accepts groups of phones, and tells you the similarities\n'
      'within the group and the differences between the two groups.\n')
print('You can get IPA chart info, vowel info, or features.\n')
print('To just see similarities, enter the digit 0 as the first item\n'
      'in group 2.\n')

while inputting:
    print('1: Features\n'
          '2: IPA(place & manner of articulation and voicing)\n'
          '3: Vowel characteristics\n')
    choice = input().strip()
    
    if choice == '0':
        inputting = False
    
    elif choice == '1':
        compare_groups(choice)
    
    elif choice == '2':
        compare_groups(choice)
    
    elif choice == '3':
        compare_groups(choice)
    
    else:
        print('Invalid')
        continue


Welcome to my phonologizer!
It accepts groups of phones, and tells you the similarities
within the group and the differences between the two groups.

You can get IPA chart info, vowel info, or features.

To just see similarities, enter the digit 0 as the first item
in group 2.

1: Features
2: IPA(place & manner of articulation and voicing)
3: Vowel characteristics

2
Inventory:
p, b, t, d, ʈ, ɖ, c, ɟ, k, g, q, ɢ, ʔ, m, ɱ, n, ɳ, ɲ, ŋ, ɴ, ʙ, r, ʀ, ⱱ, ɾ, ɽ, ɸ, β, f, v, θ , ð, s, z, ʃ, ʒ, ʂ, ʐ, ç,  ʝ, x, ɣ, χ, ʁ, ħ, ʕ, h, ɦ, ɬ, ɮ, ʋ, ɹ, ɻ, j,  ɰ, l, ɭ, ʎ, ʟ, ʍ, w, ɥ, ʜ, ʢ, ʡ, ʡ, ɕ, ʑ, ɺ
Group 1:
Enter a letter(0 to exit): h
Enter a letter(0 to exit): l
Enter a letter(0 to exit): 0

Group2:
Enter a letter(0 to exit): 0
No similarities
1: Features
2: IPA(place & manner of articulation and voicing)
3: Vowel characteristics

2
Inventory:
p, b, t, d, ʈ, ɖ, c, ɟ, k, g, q, ɢ, ʔ, m, ɱ, n, ɳ, ɲ, ŋ, ɴ, ʙ, r, ʀ, ⱱ, ɾ, ɽ, ɸ, β, f, v, θ , ð, s, z, ʃ, ʒ, ʂ, ʐ, ç,  ʝ, x, ɣ, χ, ʁ, ħ, ʕ, h, ɦ, ɬ, ɮ, ʋ, ɹ, 

,ʃ,s
features,,
0,postalveolar,alveolar


1: Features
2: IPA(place & manner of articulation and voicing)
3: Vowel characteristics

